In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import spacy

In [2]:
# nlp = spacy.load('en_core_web_lg')
# print("Enter two space-separated words")
# words = input()
# tokens = nlp(words)
# for token in tokens:
#     # Printing the following attributes of each token.
#     # text: the word string, has_vector: if it contains
#     # a vector representation in the model, 
#     # vector_norm: the algebraic norm of the vector,
#     # is_oov: if the word is out of vocabulary.
#     print(token.text, token.has_vector, token.vector_norm, token.is_oov)
  
# token1, token2 = tokens[0], tokens[1]
  
# print("Similarity:", token1.similarity(token2))

In [3]:
df_1 = pd.read_csv("data/train.csv")
df_1.head()

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00


In [4]:
df_1.describe()

,score
count,36473.000000
mean,0.362062
std,0.258335
min,0.000000
25%,0.250000
50%,0.250000
75%,0.500000
max,1.000000


In [5]:
df_1.isnull().any()

id         False
anchor     False
target     False
context    False
score      False
dtype: bool

__Label Encoder__

In [6]:
# creating instance of labelencoder
labelencoder_anchor = LabelEncoder()
labelencoder_target = LabelEncoder()
labelencoder_context = LabelEncoder()
# Assigning numerical values and storing in another column
df_1['anchor'] = labelencoder_anchor.fit_transform(df_1['anchor'])
df_1['target'] = labelencoder_target.fit_transform(df_1['target'])
df_1['context'] = labelencoder_context.fit_transform(df_1['context'])

In [7]:
df_1.head()

,id,anchor,target,context,score
0,37d61fd2272659b1,0,45,10,0.50
1,7b9652b17b68b7a4,0,279,10,0.75
2,36d72442aefd8232,0,324,10,0.25
3,5296b0c19e1ce60e,0,8716,10,0.50
4,54c1e3b9184cb5b6,0,10543,10,0.00


__Preparing test__

In [8]:
# test = pd.read_csv("test.csv")
# test['anchor'] = labelencoder_anchor.transform(test['anchor'])
# test['target'] = labelencoder_target.transform(test['target'])
# test['context'] = labelencoder_context.transform(test['context'])
# test.head()
# test_id = np.array(test['id'])
# test.drop('id',axis=1,inplace=True)
# x_test = test

__Spliting__

In [9]:
x = df_1[['anchor','target','context']].values
y = df_1['score'].values

In [10]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=141)

__Random Forest Regressor__

In [11]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(random_state=141)
regr.fit(x_train, y_train)
y_pred = np.round(regr.predict(x_test),decimals=2)
from sklearn.metrics import mean_squared_error, mean_absolute_error
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred,squared=False)
mae = mean_absolute_error(y_test,y_pred)
print("Mean Square Error: {} \nRoot Mean Squared Error: {} \nMean Absolute Error: {} ".format(mse,rmse,mae))

Mean Square Error: 0.06470339060500822 
Root Mean Squared Error: 0.2543686116740983 
Mean Absolute Error: 0.19084353865838055 


__Feature Importance__

In [12]:
features = [df_1.columns[i] for i in range(1,4)]
feature_importance_nums = regr.feature_importances_
feature_importance_df = pd.DataFrame(feature_importance_nums).transpose()
feature_importance_df.columns = features
feature_importance_df.head()

,anchor,target,context
0,0.231136,0.623436,0.145428


__Lasso Regression__

In [13]:
from sklearn.linear_model import Lasso
lr = Lasso(alpha=0.5)
lr.fit(x_train, y_train)
y_pred = np.round(lr.predict(x_test),decimals=2)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred,squared=False)
mae = mean_absolute_error(y_test,y_pred)
print("Mean Square Error: {} \nRoot Mean Squared Error: {} \nMean Absolute Error: {} ".format(mse,rmse,mae))

Mean Square Error: 0.06716260281484189 
Root Mean Squared Error: 0.2591574865112754 
Mean Absolute Error: 0.21922683238895996 


__Prediction Submission__

In [14]:
# pd.read_csv("sample_submission.csv").head()

# df_final.head()

# df_final.to_csv("Michael_Woo_Submission.csv",index=False)

# pd.read_csv("Michael_Woo_Submission.csv")